In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#Loading Datasets
train= pd.read_csv('../data/train.csv')
stores=pd.read_csv('../data/stores.csv')
features=pd.read_csv('../data/features.csv')

In [ ]:
train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [6]:
stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [7]:
features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [10]:
train.info()
stores.info()
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Store   45 non-null     int64 
 1   Type    45 non-null     object
 2   Size    45 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        ---------

In [12]:
#Convert Dates from object datatype to Datetime
train['Date']=pd.to_datetime(train['Date'])
features['Date']=pd.to_datetime(features['Date'])

In [27]:
#Handling missing values
#Filling missing values of Markdowns with 0 which indicates there is no promo held that week
#Filling CPI and Unemployment with median to avoid bias and maintain stable, realistic data.
markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
features[markdown_cols] = features[markdown_cols].fillna(0)
features['CPI'].fillna(features['CPI'].median(), inplace=True)
features['Unemployment'].fillna(features['Unemployment'].median(), inplace=True)

/var/folders/8j/6kn8hq5d0js146rby2kt74qm0000gn/T/ipykernel_50700/2327221474.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  features['CPI'].fillna(features['CPI'].median(), inplace=True)
/var/folders/8j/6kn8hq5d0js146rby2kt74qm0000gn/T/ipykernel_50700/2327221474.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setti

In [31]:
#Extracting year, week and month from Date 
train['Year']= train['Date'].dt.year
train['Month']=train['Date'].dt.month
train['Week']=train['Date'].dt.isocalendar().week
train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Year,Month,Week
0,1,1,2010-02-05,24924.50,False,2010,2,5
1,1,1,2010-02-12,46039.49,True,2010,2,6
2,1,1,2010-02-19,41595.55,False,2010,2,7
3,1,1,2010-02-26,19403.54,False,2010,2,8
4,1,1,2010-03-05,21827.90,False,2010,3,9


,Store,Dept,Date,Weekly_Sales,IsHoliday,Year,Month,Week
0,1,1,2010-02-05,24924.50,False,2010,2,5
1,1,1,2010-02-12,46039.49,True,2010,2,6
2,1,1,2010-02-19,41595.55,False,2010,2,7
3,1,1,2010-02-26,19403.54,False,2010,2,8
4,1,1,2010-03-05,21827.90,False,2010,3,9


In [32]:
#Merging the Dataset 
merged_dataset=features.merge(stores,on='Store',how='left')
df=train.merge(merged_dataset, on=['Store','Date','IsHoliday'],how='left')
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Year,Month,Week,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2010-02-05,24924.50,False,2010,2,5,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,A,151315
1,1,1,2010-02-12,46039.49,True,2010,2,6,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,A,151315
2,1,1,2010-02-19,41595.55,False,2010,2,7,39.93,2.514,0.0,0.0,0.0,0.0,0.0,211.289143,8.106,A,151315
3,1,1,2010-02-26,19403.54,False,2010,2,8,46.63,2.561,0.0,0.0,0.0,0.0,0.0,211.319643,8.106,A,151315
4,1,1,2010-03-05,21827.90,False,2010,3,9,46.50,2.625,0.0,0.0,0.0,0.0,0.0,211.350143,8.106,A,151315


In [39]:
print(df.columns)

Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Year', 'Month',
       'Week', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2',
       'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type',
       'Size', 'Hot_Week'],
      dtype='object')


In [ ]:
#Create custom column 'Hot_Week' to check if the temperature affected the sales or store visits 
df['Hot_Week']=df['Temperature']>50